# 台灣鳥類辨識器 - 遷移式學習

使用 MobileNetV2 預訓練模型識別台灣常見八哥科鳥類

## 目標
- 識別三種八哥：白尾八哥、家八哥、林八哥
- 使用遷移式學習加快訓練速度
- 達到 85% 以上準確率

## 1. 載入必要套件

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
import matplotlib.pyplot as plt
import numpy as np
from PIL import Image
import os

print(f"TensorFlow 版本: {tf.__version__}")
print(f"Keras 版本: {keras.__version__}")

## 2. 設定參數

In [ ]:
# 資料路徑
TRAIN_DIR = '../data/train'
TEST_DIR = '../data/test'
MODEL_SAVE_PATH = '../models/bird_classifier.keras'

# 圖片參數
IMG_SIZE = 224  # MobileNetV2 輸入大小
BATCH_SIZE = 32

# 訓練參數
EPOCHS = 20
LEARNING_RATE = 0.0001

# 類別（依照資料夾名稱）
CLASSES = ['白尾八哥', '家八哥', '林八哥']
NUM_CLASSES = len(CLASSES)

print(f"類別數量: {NUM_CLASSES}")
print(f"類別名稱: {CLASSES}")

## 3. 準備資料生成器（Data Augmentation）

In [ ]:
# 訓練資料生成器 - 包含資料擴增
train_datagen = ImageDataGenerator(
    rescale=1./255,              # 正規化到 0-1
    rotation_range=20,           # 隨機旋轉 20 度
    width_shift_range=0.2,       # 水平平移
    height_shift_range=0.2,      # 垂直平移
    shear_range=0.2,             # 剪切變換
    zoom_range=0.2,              # 隨機縮放
    horizontal_flip=True,        # 水平翻轉
    fill_mode='nearest',         # 填充模式
    validation_split=0.2         # 20% 作為驗證集
)

# 測試資料生成器 - 只做正規化
test_datagen = ImageDataGenerator(
    rescale=1./255
)

## 4. 載入訓練和驗證資料

In [ ]:
# 訓練集
train_generator = train_datagen.flow_from_directory(
    TRAIN_DIR,
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='training'
)

# 驗證集
validation_generator = train_datagen.flow_from_directory(
    TRAIN_DIR,
    target_size=(IMG_SIZE, IMG_SIZE),
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    subset='validation'
)

# 測試集（如果有的話）
if os.path.exists(TEST_DIR):
    test_generator = test_datagen.flow_from_directory(
        TEST_DIR,
        target_size=(IMG_SIZE, IMG_SIZE),
        batch_size=BATCH_SIZE,
        class_mode='categorical'
    )
    print(f"\n測試集圖片數量: {test_generator.samples}")

print(f"\n訓練集圖片數量: {train_generator.samples}")
print(f"驗證集圖片數量: {validation_generator.samples}")
print(f"\n類別索引對應: {train_generator.class_indices}")

## 5. 視覺化部分訓練資料

In [ ]:
# 取得一個批次的資料來視覺化
sample_images, sample_labels = next(train_generator)

# 建立反向對應字典（索引 -> 類別名稱）
class_names = {v: k for k, v in train_generator.class_indices.items()}

# 繪製前 9 張圖片
plt.figure(figsize=(15, 15))
for i in range(min(9, len(sample_images))):
    plt.subplot(3, 3, i + 1)
    plt.imshow(sample_images[i])
    label_idx = np.argmax(sample_labels[i])
    plt.title(f"{class_names[label_idx]}")
    plt.axis('off')
plt.tight_layout()
plt.show()

## 6. 建立遷移式學習模型

使用 MobileNetV2 作為基礎模型

In [ ]:
# 載入 MobileNetV2 預訓練模型（不包含頂層）
base_model = MobileNetV2(
    input_shape=(IMG_SIZE, IMG_SIZE, 3),
    include_top=False,           # 不包含最後的分類層
    weights='imagenet'           # 使用 ImageNet 預訓練權重
)

# 凍結基礎模型的權重（不訓練）
base_model.trainable = False

print("MobileNetV2 基礎模型載入成功！")
print(f"基礎模型總層數: {len(base_model.layers)}")

## 7. 添加自訂分類層

In [ ]:
# 建立完整模型
inputs = keras.Input(shape=(IMG_SIZE, IMG_SIZE, 3))

# MobileNetV2 特徵提取
x = base_model(inputs, training=False)

# 全局平均池化
x = GlobalAveragePooling2D()(x)

# 全連接層
x = Dense(256, activation='relu')(x)
x = Dropout(0.5)(x)  # Dropout 防止過擬合

# 輸出層
outputs = Dense(NUM_CLASSES, activation='softmax')(x)

# 組合成完整模型
model = Model(inputs, outputs)

print("\n完整模型建立成功！")

## 8. 編譯模型

In [ ]:
model.compile(
    optimizer=Adam(learning_rate=LEARNING_RATE),
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

print("模型編譯完成！")

## 9. 查看模型架構

In [ ]:
model.summary()

## 10. 開始訓練模型

In [ ]:
# 設定回調函數
callbacks = [
    keras.callbacks.EarlyStopping(
        monitor='val_loss',
        patience=5,
        restore_best_weights=True
    ),
    keras.callbacks.ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.5,
        patience=3,
        min_lr=1e-7
    )
]

# 開始訓練
history = model.fit(
    train_generator,
    validation_data=validation_generator,
    epochs=EPOCHS,
    callbacks=callbacks
)

print("\n訓練完成！")

## 11. 視覺化訓練過程

In [ ]:
# 繪製訓練和驗證的準確率
plt.figure(figsize=(14, 5))

# 準確率
plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'], label='訓練準確率')
plt.plot(history.history['val_accuracy'], label='驗證準確率')
plt.title('模型準確率')
plt.xlabel('Epoch')
plt.ylabel('準確率')
plt.legend()
plt.grid(True)

# 損失
plt.subplot(1, 2, 2)
plt.plot(history.history['loss'], label='訓練損失')
plt.plot(history.history['val_loss'], label='驗證損失')
plt.title('模型損失')
plt.xlabel('Epoch')
plt.ylabel('損失')
plt.legend()
plt.grid(True)

plt.tight_layout()
plt.show()

# 顯示最終結果
final_train_acc = history.history['accuracy'][-1]
final_val_acc = history.history['val_accuracy'][-1]
print(f"\n最終訓練準確率: {final_train_acc:.4f}")
print(f"最終驗證準確率: {final_val_acc:.4f}")

## 12. 評估模型（如果有測試集）

In [ ]:
if os.path.exists(TEST_DIR):
    test_loss, test_accuracy = model.evaluate(test_generator)
    print(f"\n測試集準確率: {test_accuracy:.4f}")
    print(f"測試集損失: {test_loss:.4f}")
else:
    print("\n未找到測試集資料夾")

## 13. 儲存模型

In [ ]:
# 建立模型資料夾（如果不存在）
os.makedirs(os.path.dirname(MODEL_SAVE_PATH), exist_ok=True)

# 儲存模型
model.save(MODEL_SAVE_PATH)
print(f"\n模型已儲存至: {MODEL_SAVE_PATH}")

# 也可以儲存成 H5 格式（舊版本相容）
h5_path = MODEL_SAVE_PATH.replace('.keras', '.h5')
model.save(h5_path)
print(f"模型也儲存成 H5 格式: {h5_path}")

## 14. 測試預測功能

In [ ]:
def predict_bird(image_path, model, class_names):
    """
    預測單張圖片
    """
    # 載入並預處理圖片
    img = Image.open(image_path).convert('RGB')
    img = img.resize((IMG_SIZE, IMG_SIZE))
    img_array = np.array(img) / 255.0
    img_array = np.expand_dims(img_array, axis=0)
    
    # 預測
    predictions = model.predict(img_array, verbose=0)
    predicted_class_idx = np.argmax(predictions[0])
    confidence = predictions[0][predicted_class_idx]
    
    # 顯示結果
    plt.figure(figsize=(10, 4))
    
    # 顯示圖片
    plt.subplot(1, 2, 1)
    plt.imshow(img)
    plt.title(f"預測: {class_names[predicted_class_idx]}\n信心度: {confidence:.2%}")
    plt.axis('off')
    
    # 顯示機率分布
    plt.subplot(1, 2, 2)
    plt.bar(range(NUM_CLASSES), predictions[0])
    plt.xticks(range(NUM_CLASSES), [class_names[i] for i in range(NUM_CLASSES)], rotation=45)
    plt.ylabel('機率')
    plt.title('各類別預測機率')
    plt.ylim([0, 1])
    
    plt.tight_layout()
    plt.show()
    
    return class_names[predicted_class_idx], confidence

print("預測函數已定義！")

## 15. 測試預測（使用驗證集的圖片）

In [ ]:
# 從驗證集隨機選擇一張圖片測試
import random

# 取得所有圖片路徑
all_images = []
for class_name in CLASSES:
    class_dir = os.path.join(TRAIN_DIR, class_name)
    if os.path.exists(class_dir):
        images = [os.path.join(class_dir, f) for f in os.listdir(class_dir) 
                 if f.lower().endswith(('.png', '.jpg', '.jpeg'))]
        all_images.extend(images)

if all_images:
    # 隨機選擇一張圖片
    test_image_path = random.choice(all_images)
    print(f"測試圖片: {test_image_path}\n")
    
    # 預測
    predicted_class, confidence = predict_bird(test_image_path, model, class_names)
    print(f"\n預測結果: {predicted_class}")
    print(f"信心度: {confidence:.2%}")
else:
    print("找不到圖片進行測試")

## 16. 完成！

模型訓練完成！接下來可以：
1. 執行 `streamlit run src/app.py` 啟動網頁應用
2. 上傳圖片進行鳥類辨識
3. 部署到 Streamlit Cloud 分享給他人使用

### 可能的改進方向：
- 增加更多訓練資料
- 微調（fine-tune）基礎模型的部分層
- 嘗試其他預訓練模型（如 ResNet, EfficientNet）
- 調整超參數（學習率、批次大小等）
- 增加更多資料擴增技術